In [21]:
import pandas as pd
from src.config import get_interim_dir
import seaborn as sns
from random import choices
from pyod.models.mcd import MCD
from pyod.models.ecod import ECOD
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [22]:
all_data = {}
for t in ["T1", "T2", "T3"]:
    all_data[t] = pd.read_pickle(get_interim_dir() + "agg_pca_analysis_{}.pkl".format(t))

In [23]:
X = all_data["T2"]["X"]

In [24]:
X = all_data[t]["X"]
profiles=[]
def generate_bootstrap_samples(x):
    repair_outcomes = x.index.to_list()
    outcome_frequencies = x.values
    boostrapped_outcomes = pd.Series(choices(repair_outcomes, outcome_frequencies, k=1000)).value_counts()
    boostrapped_outcomes = boostrapped_outcomes/boostrapped_outcomes.sum()
    # profiles.loc[x.name, boostrapped_outcomes.index] = boostrapped_outcomes.values
    profiles.append(pd.DataFrame({
        "Gene": x.name,
        "Frequency" : boostrapped_outcomes.values,
        "Outcome": boostrapped_outcomes.index.to_list(),
        }))

X.apply(generate_bootstrap_samples, axis=1)
profiles = pd.concat(profiles)
profiles = profiles.pivot(index=["Gene"], columns=["Outcome"], values="Frequency")
profiles.sum(axis=1).head()

Gene
0610009B22Rik    1.0
0610010K14Rik    1.0
0610030E20Rik    1.0
0610040J01Rik    1.0
1110004F10Rik    1.0
dtype: float64

In [25]:
# need to deal with zero values
print("{:.2f} of our data is zero".format((profiles.isnull()).sum().sum()/(profiles.shape[0] * profiles.shape[1])))

0.36 of our data is zero


In [26]:
# before applyiing log transformations, we need to add random uniform values between 0.1*DL and DL to our zero values
# where DL is the "Detection Limit". In our case, we have 1000 reads. This means that repair outcomes that occur with less than 1/1000 frequency are unlikely
# to be represented. Thus we choose 0.001 to be our DL
# see https://reader.elsevier.com/reader/sd/pii/S0169743921000162?token=17AE18CDF91419243B4FE3BA84D1010DC2FBE394617633976872735270CE3EB70118E1FD4D02151635B72A2D7EE97ADC&originRegion=eu-west-1&originCreation=20221017112946
# "Comparison of zero replacement strategies for compositional data with large numbers of zeros", 2021

DL = 0.001
M = len(profiles.index)
N = len(profiles.columns)
ran = pd.DataFrame(np.random.uniform(low=0.1*DL, high=DL, size=(M,N)), columns=profiles.columns, index=profiles.index)

ran.update(profiles)
ran.sum(axis=1)
profiles_imp = ran
profiles_imp = profiles_imp.div(profiles_imp.sum(axis=1), axis=0)
print("{:.2f} of our data is zero".format((profiles_imp.isnull()).sum().sum()/(profiles_imp.shape[0] * profiles_imp.shape[1])))

0.00 of our data is zero


In [27]:
# need to perform isometric log ratio transform]
# https://onlinelibrary.wiley.com/doi/epdf/10.1002/env.966
# https://www.youtube.com/watch?v=fQPCeV4MUe4&t=2s
# https://www.youtube.com/watch?v=WQ29fPfOngA&list=PLh35GyCXlQaSLdi-kMCNohZSEKNDVSO-I&index=1


from skbio.stats.composition import ilr, clr

print(profiles_imp.shape)
Z = ilr(profiles_imp.values) # Eq 5
print(Z.shape)
Y = clr(profiles_imp.values)

(17884, 88)
(17884, 87)


In [28]:
D = Y.shape[1]
V = np.array([np.sqrt(i/(i+1)) * np.concatenate((np.repeat(1/i, i), np.array([-1]), np.repeat(0, D - i - 1))) for i in range(1, D)])
V.shape

(87, 88)

In [29]:
from sklearn.covariance import MinCovDet
from sklearn.covariance import EmpiricalCovariance
from numpy.linalg import svd


cov = MinCovDet(random_state=0).fit(Z)
# cov = EmpiricalCovariance().fit(Z)
T = cov.location_
C = cov.covariance_

G, L, G_inv = svd(C) 
# G = eigenvectors
# L = eigenvalues

Z_t = np.dot(Z - T, G)

In [30]:
print("sanity checks...")
print(np.all(np.equal(np.round_(Y, 4), np.round_(np.dot(Z, V), 4))))
print(np.all(np.equal(np.round_(Z, 4), np.round_(np.dot(Y, np.transpose(V)), 4))))
print(np.all(np.equal(np.round(G_inv, 4), np.round(np.linalg.inv(G), 4))))

sanity checks...
True
True
True


In [31]:
Y_t = np.dot(Z_t, V)

In [32]:
C_y = np.dot(np.dot(np.transpose(V), (G * L * G_inv)), V)

In [33]:
G_y = np.dot(G, V)
np.sum(np.round(np.dot(G_y, np.transpose(G_y)), 4)) == (D-1)

True

In [34]:
from scipy.stats import chi2

# calculate mahalanobis distances
distances = cov.mahalanobis(Z)

# select significance threshold
pvalues = 1 - chi2.cdf(distances, Z.shape[1] - 1)
pvalues


array([0.96181325, 0.9436936 , 0.8686101 , ..., 0.13655928, 0.89406776,
       0.44464375])

In [35]:
from statsmodels.stats.multitest import fdrcorrection

rejected, corrected_pvalues = fdrcorrection(pvalues)
sum(rejected)

2050

In [66]:
df = pd.DataFrame({
    "pvalue": pvalues,
    "rejected": rejected,
    "corrected_pvalues": corrected_pvalues,
}, index=profiles.index)
df.sort_values("pvalue").index.get_loc("Gzmg")

2026

### What is the effect of the different types of transformation?


In [15]:
import pandas as pd
from src.config import get_interim_dir

X = pd.read_pickle(get_interim_dir() + "genewise_repair_outcome_profiles.pkl")
counts = X.iloc[:,-1:]
X = X.iloc[:,:-1]
counts.head()

Feature                counts
Target Gene                  
T1     0610009B22Rik  31948.0
       0610010K14Rik  42869.0
       0610030E20Rik  16913.0
       0610040J01Rik  12071.0
       1110004F10Rik  12516.0

In [5]:
X[X>0].min().min()

7.288895367906994e-06

In [37]:
X_unscaled = (X * counts.values)
X_unscaled

Feature               1bp insertion - A, 0  1bp insertion - C, 0  \
Target Gene                                                        
T1     0610009B22Rik           2097.620625             91.407758   
       0610010K14Rik           2607.985635             68.542186   
       0610030E20Rik           1027.397547             33.145610   
       0610040J01Rik            717.188131             10.438430   
       1110004F10Rik            718.572098             15.409608   
...                                    ...                   ...   
T3     Zzef1                     22.120295            142.125527   
       Zzz3                      68.055249            172.810698   
       a                         23.475087            168.117824   
       ccdc198                   29.251128            127.000407   
       mei-04                    50.230168            102.337081   

Feature               1bp insertion - G, 0  1bp insertion - T, 0  \
Target Gene                                                        
T1     0610009B22Rik            226.816128            112.804050   
       0610010K14Rik            248.428514            148.571474   
       0610030E20Rik            127.760163             68.264679   
       0610040J01Rik            135.881278             25.147894   
       1110004F10Rik             77.694863             46.389189   
...                                    ...                   ...   
T3     Zzef1                   7028.660511             48.197997   
       Zzz3                    7338.814185            122.759933   
       a                       6000.243065             44.902977   
       ccdc198                 6397.007671             68.364319   
       mei-04                  4258.596219             34.421114   

Feature               >1bp insertion, 0  Deletion 0bp microhomology, 1  \
Target Gene                                                              
T1     0610009B22Rik         727.717777                      18.302336   
       0610010K14Rik        1220.085186                      16.388530   
       0610030E20Rik         405.934051                       6.105776   
       0610040J01Rik         338.033724                      14.780816   
       1110004F10Rik         237.571709                       0.000000   
...                                 ...                            ...   
T3     Zzef1                 218.721965                     756.983279   
       Zzz3                  227.264071                     765.796937   
       a                     243.114732                     655.735151   
       ccdc198               235.565318                     727.220514   
       mei-04                149.648627                     443.017442   

Feature               Deletion 0bp microhomology, 10  \
Target Gene                                            
T1     0610009B22Rik                       90.436022   
       0610010K14Rik                       95.175620   
       0610030E20Rik                       41.036029   
       0610040J01Rik                       21.224807   
       1110004F10Rik                       33.737009   
...                                              ...   
T3     Zzef1                              156.130604   
       Zzz3                               105.187039   
       a                                   95.703609   
       ccdc198                             68.765969   
       mei-04                              94.419473   

Feature               Deletion 0bp microhomology, 11  \
Target Gene                                            
T1     0610009B22Rik                      223.377588   
       0610010K14Rik                      163.374350   
       0610030E20Rik                      118.611070   
       0610040J01Rik                       58.251516   
       1110004F10Rik                       27.187348   
...                                              ...   
T3     Zzef1                               66.190867   
       Zzz3                       

In [61]:
import numpy as np

def alr(X):
    X_r = X_unscaled.iloc[:,0]
    X_2 = X_unscaled.iloc[:,1:]
    X_3 = X_2.div(X_r, axis=0)
    return np.log2(X_3)

print(X.iloc[:,1:].iloc[:5,4])

print(alr(X_unscaled).iloc[:5,4])

print(alr(X).iloc[:5,4])

Target  Gene         
T1      0610009B22Rik    0.000573
        0610010K14Rik    0.000382
        0610030E20Rik    0.000361
        0610040J01Rik    0.001224
        1110004F10Rik    0.000000
Name: Deletion 0bp microhomology, 1, dtype: float64
Target  Gene         
T1      0610009B22Rik   -6.840582
        0610010K14Rik   -7.314106
        0610030E20Rik   -7.394604
        0610040J01Rik   -5.600554
        1110004F10Rik        -inf
Name: Deletion 0bp microhomology, 1, dtype: float64
Target  Gene         
T1      0610009B22Rik   -6.840582
        0610010K14Rik   -7.314106
        0610030E20Rik   -7.394604
        0610040J01Rik   -5.600554
        1110004F10Rik        -inf
Name: Deletion 0bp microhomology, 1, dtype: float64


In [70]:
X

Feature               1bp insertion - A, 0  1bp insertion - C, 0  \
Target Gene                                                        
T1     0610009B22Rik              0.065657              0.002861   
       0610010K14Rik              0.060836              0.001599   
       0610030E20Rik              0.060746              0.001960   
       0610040J01Rik              0.059414              0.000865   
       1110004F10Rik              0.057412              0.001231   
...                                    ...                   ...   
T3     Zzef1                      0.000782              0.005023   
       Zzz3                       0.002174              0.005521   
       a                          0.000965              0.006913   
       ccdc198                    0.001065              0.004625   
       mei-04                     0.002603              0.005304   

Feature               1bp insertion - G, 0  1bp insertion - T, 0  \
Target Gene                                                        
T1     0610009B22Rik              0.007100              0.003531   
       0610010K14Rik              0.005795              0.003466   
       0610030E20Rik              0.007554              0.004036   
       0610040J01Rik              0.011257              0.002083   
       1110004F10Rik              0.006208              0.003706   
...                                    ...                   ...   
T3     Zzef1                      0.248424              0.001704   
       Zzz3                       0.234467              0.003922   
       a                          0.246731              0.001846   
       ccdc198                    0.232949              0.002490   
       mei-04                     0.220721              0.001784   

Feature               >1bp insertion, 0  Deletion 0bp microhomology, 1  \
Target Gene                                                              
T1     0610009B22Rik           0.022778                       0.000573   
       0610010K14Rik           0.028461                       0.000382   
       0610030E20Rik           0.024001                       0.000361   
       0610040J01Rik           0.028004                       0.001224   
       1110004F10Rik           0.018981                       0.000000   
...                                 ...                            ...   
T3     Zzef1                   0.007731                       0.026755   
       Zzz3                    0.007261                       0.024466   
       a                       0.009997                       0.026964   
       ccdc198                 0.008578                       0.026482   
       mei-04                  0.007756                       0.022961   

Feature               Deletion 0bp microhomology, 10  \
Target Gene                                            
T1     0610009B22Rik                        0.002831   
       0610010K14Rik                        0.002220   
       0610030E20Rik                        0.002426   
       0610040J01Rik                        0.001758   
       1110004F10Rik                        0.002696   
...                                              ...   
T3     Zzef1                                0.005518   
       Zzz3                                 0.003361   
       a                                    0.003935   
       ccdc198                              0.002504   
       mei-04                               0.004894   

Feature               Deletion 0bp microhomology, 11  \
Target Gene                                            
T1     0610009B22Rik                        0.006992   
       0610010K14Rik                        0.003811   
       0610030E20Rik                        0.007013   
       0610040J01Rik                        0.004826   
       1110004F10Rik                        0.002172   
...                                              ...   
T3     Zzef1                                0.002339   
       Zzz3                       